In [ ]:
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import keras
from sklearn.model_selection import train_test_split

Plot params

In [ ]:
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'FreeSerif'
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['lines.markersize'] = 12
plt.rcParams['xtick.labelsize'] = 24
plt.rcParams['ytick.labelsize'] = 24
plt.rcParams['legend.fontsize'] = 24
plt.rcParams['axes.titlesize'] = 36
plt.rcParams['axes.labelsize'] = 24

In [ ]:
data_train = pd.read_csv("./data_sets/train.csv")
data_test = pd.read_csv("./data_sets/test.csv")
(X,y) = (data_train.drop(columns=["label"]), data_train["label"])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=17)
X_train = X_train.astype('float32') 
X_test = X_test.astype('float32')
X_train = X_train.values.reshape(X_train.shape[0], 1, 28, 28)
X_test = X_test.values.reshape(X_test.shape[0], 1, 28, 28)
X_train /= 255 # Normalise data to [0, 1] range
X_test /= 255 # Normalise data to [0, 1] range

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
f = plt.figure(figsize=(10, 10))

ax = plt.subplot(2, 2, 1)
ax.imshow(X_train[0].reshape([28, 28]))

ax = plt.subplot(2, 2, 2)
i = np.random.randint(0, X_train.shape[0], 1)
ax.imshow(X_train[i].reshape([28, 28]))

ax = plt.subplot(2, 2, 3)
i = np.random.randint(0, X_train.shape[0], 1)
ax.imshow(X_train[i].reshape([28, 28]))

ax = plt.subplot(2, 2, 4)
i = np.random.randint(0, X_train.shape[0], 1)
ax.imshow(X_train[i].reshape([28, 28]))

plt.show()

In [ ]:
batch_size = 128 # in each iteration, we consider 128 training examples at once
num_epochs = 15 # we iterate twenty times over the entire training set
hidden_size = 512 # there will be 512 neurons in both hidden layers

In [ ]:
model = Sequential()
model.add(Conv2D(30, (5, 5), input_shape=(1, 28, 28), activation='relu',data_format='channels_first'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(60, (3, 3), activation='relu',data_format='channels_first'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(120, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(60, activation='sigmoid'))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='Adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=num_epochs,
                    verbose=1,
                    validation_split=0.3)

In [ ]:
model.evaluate(X_test,y_test, verbose=1)

Draw Plots of errors

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')
plt.grid()
plt.show()

# Plot training & validation loss values
plt.figure(figsize=(10,7))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper right')
plt.grid()
plt.show()

learning plots

In [ ]:
set_sizes = np.arange(1000, 10000, 1000)
set_sizes = np.hstack((set_sizes, list(range(10, 100, 10))))
set_sizes = np.hstack((set_sizes, list(range(100, 1000, 100))))
set_sizes = np.sort(set_sizes)
accuracy_metric = []

In [ ]:
for size in set_sizes:
    print(size)
    used_indices = np.random.choice(np.arange(X_train.shape[0]), size, replace=False)
    model.fit(X_train[used_indices, :], y_train[used_indices, :],
                    batch_size=batch_size,
                    epochs=num_epochs,
                    verbose=0,
                    validation_split=0.3)
    accuracy_metric.append(model.evaluate(X_test,y_test, verbose=1))
    print(accuracy_metric[-1])

In [ ]:
accuracy_loss = np.asarray(accuracy_metric)

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(set_sizes, accuracy_loss[:, 1])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Set size')
plt.legend(['Train'], loc='upper right')
plt.grid()
plt.show()

# Plot training & validation loss values
plt.figure(figsize=(10,7))
plt.plot(set_sizes, accuracy_loss[:, 0])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Set Size')
plt.legend(['Train'], loc='upper right')
plt.grid()
plt.show()

Adding a noize

In [ ]:
def get_noisy_x(X_train, mean, stddev):
    n_imgs = X_train.shape[0]
    n_chan = X_train.shape[1]
    n_rows = X_train.shape[2]
    n_cols = X_train.shape[3]
    if stddev == 0:
        noise = np.zeros((n_imgs, n_chan, n_rows, n_cols))
    else:
        noise = np.random.normal(mean, stddev/255., 
                                     (n_imgs, n_chan, n_rows, n_cols))
    noisy_X = X_train + noise
    noisy_X = np.clip(noisy_X, 0., 1.)
    return noisy_X

In [ ]:
f = plt.figure(figsize=(10, 10))
for i in range(1, 5):
    ax = plt.subplot(2, 2, i)
    ax.imshow(get_noisy_x(X_train, mean=0, stddev=10*(i - 1))[0].reshape([28, 28]))
plt.show()

In [ ]:
noises = [x for x in range(0, 110, 10)]
histories = []

In [ ]:
for noise in noises:
    print(noise)
    histories.append(model.fit(get_noisy_x(X_train, mean=0, stddev=noise), y_train,
                    batch_size=batch_size,
                    epochs=num_epochs,
                    verbose=0,
                    validation_split=0.3))

Draw plots

In [ ]:
plt.figure(figsize=(20, 15))
i = 0
for history in histories:  
    plt.plot(history.history['acc'], label="Noise =" + str(noises[i]/ 255))
    i+= 1
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.title('Model accuracy')
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(20, 15))
i = 0
for history in histories:  
    plt.plot(history.history['loss'], label="Noise =" + str(noises[i]/255))
    i+= 1
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.title('Model loss')
plt.legend()
plt.grid()
plt.show()